In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

In [ ]:
AvanaGuideMap = tc.get(name='chronos2-0test3-internal-d93b', version=1, file='AvanaGuideMap')

In [ ]:
AvanaGuideMap.head()

In [ ]:
plt.hist(AvanaGuideMap.nAlignments, bins=20)

In [ ]:
# AvanaLogfoldChange = tc.get(name='chronos2-0test3-internal-d93b', version=1, file='AvanaLogfoldChange')

In [ ]:
# AvanaLogfoldChange.head()

In [ ]:
# AvanaLogfoldChange.shape, AvanaGuideMap.shape

In [ ]:
# np.intersect1d(AvanaGuideMap.loc[AvanaGuideMap.nAlignments==1, :].sgRNA, AvanaLogfoldChange.index).shape

In [ ]:
# AvanaGuideMap.head()

In [ ]:
# AvanaLogfoldChange.head()

In [ ]:
ScreenNaiveGeneScore = tc.get(name='chronos2-0test3-internal-d93b', version=1, file='ScreenNaiveGeneScore')

In [ ]:
ScreenNaiveGeneScore.head()

In [ ]:
AvanaGuideMap.loc[AvanaGuideMap.nAlignments==1, 'Gene'].unique().shape

In [ ]:
OmicsCNSegmentsProfile = tc.get(name='internal-23q2-1e49', version=59, file='OmicsCNSegmentsProfile')


In [ ]:
OmicsCNSegmentsProfile.head()

In [ ]:
OmicsCNGene = tc.get(name='internal-23q2-1e49', version=59, file='OmicsCNGene')

In [ ]:
OmicsExpressionProteinCodingGenesTPMLogp1 = tc.get(name='internal-23q2-1e49', version=59, file='OmicsExpressionProteinCodingGenesTPMLogp1')

In [ ]:
OmicsExpressionProteinCodingGenesTPMLogp1.head()

In [ ]:
overlap_exp_cn = np.intersect1d(OmicsCNGene.index.values, OmicsExpressionProteinCodingGenesTPMLogp1.index.values)
overlap_genes = np.intersect1d(OmicsCNGene.columns.values, OmicsExpressionProteinCodingGenesTPMLogp1.columns.values)
expression = OmicsExpressionProteinCodingGenesTPMLogp1.loc[overlap_exp_cn, overlap_genes]

In [ ]:
cnv = OmicsCNGene.loc[overlap_exp_cn, overlap_genes]

In [ ]:
cnv.shape, expression.shape

In [ ]:
import pandas as pd
from scipy.stats import pearsonr
pearson_exp_cnv = []
for sample in cnv.index:
    exp_cnv = pd.DataFrame({"CNV": cnv.loc[sample], "EXP": expression.loc[sample]})
    #print(exp_cnv.shape)
    pearson_exp_cnv.append(exp_cnv.dropna().corr().values[1][0])

In [ ]:
plt.hist(pearson_exp_cnv)

In [ ]:
OmicsCNGene.loc['ACH-000917', :]

In [ ]:
OmicsProfiles = tc.get(name='public-23q2-19de', file='OmicsProfiles')

In [ ]:
OmicsProfiles.head()

In [ ]:
OmicsProfiles.ModelID.unique().shape, OmicsProfiles.ModelID.shape

In [ ]:
OmicsProfiles.loc[OmicsProfiles.ModelID== 'ACH-001131', :]

In [ ]:
OmicsProfiles.loc[OmicsProfiles.ModelID== 'ACH-000277', :]

In [ ]:
OmicsDefaultModelProfiles = tc.get(name='internal-23q2-1e49', version=59, file='OmicsDefaultModelProfiles')

In [ ]:
OmicsDefaultModelProfiles.head()

In [ ]:
CRISPRInferredCommonEssentials = tc.get(name='internal-22q4-56d4', version=93, file='CRISPRInferredCommonEssentials')

In [ ]:
CRISPRInferredCommonEssentials.head()

In [ ]:
ScreenSequenceMap = tc.get(name='internal-22q4-56d4', version=93, file='ScreenSequenceMap')

In [ ]:
ScreenSequenceMap.loc[ScreenSequenceMap.ModelID.isin(['ACH-001799', 'ACH-001819']), :]

In [ ]:
CRISPRInferredCommonEssentials.shape

In [ ]:
ScreenNaiveGeneScore.shape

In [ ]:
ScreenNaiveGeneScore = ScreenNaiveGeneScore.loc[:, np.setdiff1d(ScreenNaiveGeneScore.columns.values, CRISPRInferredCommonEssentials.Essentials.values)]

In [ ]:
ScreenNaiveGeneScore = ScreenNaiveGeneScore.loc[:, np.intersect1d(ScreenNaiveGeneScore.columns, AvanaGuideMap.loc[AvanaGuideMap.nAlignments==1, 'Gene'].unique())]

In [ ]:
ScreenNaiveGeneScore.head()

In [ ]:
OmicsCNGene.shape

In [ ]:
OmicsCNGene.head()

In [ ]:
OmicsCNGene.loc['ACH-000277', :].shape

In [ ]:
OmicsCNGene.loc['ACH-000277',:]

In [ ]:
OmicsCNGene.index.unique().shape, OmicsCNGene.shape

In [ ]:
ScreenSequenceMap.head()

In [ ]:
import pandas as pd
import seaborn as sns

def pl_scatterplot_boxplot(ACH_id, cnv_cutoff=2):
    screen_id = ScreenSequenceMap.loc[ScreenSequenceMap.ModelID.isin([ACH_id]), :].ScreenID.unique()[0]
    print(screen_id)
    overlap_index = np.intersect1d(ScreenNaiveGeneScore.loc[screen_id, :].dropna().index, OmicsCNGene.loc[ACH_id, :].dropna().index)
    data = pd.DataFrame({'CN': OmicsCNGene.loc[ACH_id, :].dropna().loc[overlap_index], 
                         'ScreenNaiveScore': ScreenNaiveGeneScore.loc[screen_id, :].dropna().loc[overlap_index]})
    print(data.head())
    data.loc[:, 'ScreenNaiveScore'] -= data.loc[:, 'ScreenNaiveScore'].mean()
    # Filter common essential genes
    # Filter high copy number variation genes > 4
    data = data.loc[data.CN>=cnv_cutoff, :]
    data.loc[:, 'CNquantile'] = pd.cut(data.CN, 5, precision=1)
    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(9.3, 5)
    sns.scatterplot(x='CN', y='ScreenNaiveScore', size=1, data=data, ax=ax[0])
    sns.boxplot(x='CNquantile', y='ScreenNaiveScore', data=data, ax=ax[1] )
    ax[1].tick_params(axis='x', rotation=90)

In [ ]:
import pandas as pd
import seaborn as sns

def eval_cnv_expr_cor(ACH_id, cnv_cutoff: float =2):
    try:
        screen_id = ScreenSequenceMap.loc[ScreenSequenceMap.ModelID.isin([ACH_id]), :].ScreenID.unique()[0]
        overlap_index = np.intersect1d(ScreenNaiveGeneScore.loc[screen_id, :].dropna().index, OmicsCNGene.loc[ACH_id, :].dropna().index)
        data = pd.DataFrame({'CN': OmicsCNGene.loc[ACH_id, :].dropna().loc[overlap_index], 
                             'ScreenNaiveScore': ScreenNaiveGeneScore.loc[screen_id, :].dropna().loc[overlap_index]})
        data = data.loc[data.CN>=cnv_cutoff, :]    
        return data.corr().values[1][0]
    except:
        return np.nan

In [ ]:
for sample in OmicsCNGene.var(axis=1).sort_values(ascending=False).index[:5]:
    try:
        pl_scatterplot_boxplot(sample, cnv_cutoff=2)
        plt.title(sample)
    except:
        print(sample)
    

In [ ]:
plt.hist(OmicsCNGene.var(axis=1))

In [ ]:
crispr_cnv = []
cutoffs = [1, 2, 3, 4]
for cutoff in cutoffs:
    for sample in OmicsCNGene.var(axis=1).sort_values(ascending=False).index[:50]:
        crispr_cnv.append(eval_cnv_expr_cor(sample, cnv_cutoff=cutoff))
    plt.figure()
    plt.hist(crispr_cnv)    
    plt.title(cutoff)

In [ ]:
OmicsProfiles = tc.get(name='internal-23q2-1e49', version=59, file='OmicsProfiles')

In [ ]:
OmicsProfiles.head()

In [ ]:
merged_gene_cn_profile = tc.get(name='cn-latest-d8d4', version=40, file='merged_gene_cn_profile')

In [ ]:
merged_gene_cn_profile.head()

In [ ]:
OmicsProfiles.ProfileID.isin(merged_gene_cn_profile.index).sum()

In [ ]:
OmicsProfiles.ProfileID.unique().shape

In [ ]:
merged_gene_cn_profile.index.unique().shape

In [ ]:
OmicsProfiles.loc[~OmicsProfiles.ProfileID.isin(merged_gene_cn_profile.index)].Datatype.unique()

In [ ]:
OmicsDefaultModelProfiles.head()

In [ ]:
(~merged_gene_cn_profile.index.isin(OmicsProfiles.ProfileID)).sum()

In [ ]:
(~merged_gene_cn_profile.index.isin(OmicsDefaultModelProfiles.ProfileID)).sum() 

In [ ]:
protocol_map_dict = dict(zip(OmicsProfiles.ProfileID, OmicsProfiles.Datatype))
new_cnv_index = [protocol_map_dict.get(key, 'NA') for key in merged_gene_cn_profile.index]

In [ ]:
model_map_dict = dict(zip(OmicsProfiles.ProfileID, OmicsProfiles.ModelID))
new_model = [model_map_dict.get(key, 'NA') for key in merged_gene_cn_profile.index]

In [ ]:
merged_gene_cn_profile.loc[:, 'platform'] = new_cnv_index

In [ ]:
merged_gene_cn_profile.loc[:, 'ModelID'] = new_model

In [ ]:
merged_gene_cn_profile.head()

In [ ]:
# from depmap_omics_upload import tracker
# mytracker = tracker.SampleTracker(gumbo_env='production')
# pr_with_oncotree = mytracker.add_model_cols_to_prtable(cols=["ModelID", "depmap_model_type"])
# pr_with_oncotree.to_csv("all_meta_oncotree.tsv", sep='\t')

In [ ]:
assert (merged_gene_cn_profile.platform=='NA').sum() == (merged_gene_cn_profile.ModelID == 'NA').sum()

In [ ]:
merged_gene_cn_profile_nonull = merged_gene_cn_profile.loc[merged_gene_cn_profile.platform!='NA', :]

In [ ]:
merged_gene_cn_profile_nonull.head()

In [ ]:
merged_gene_cn_profile_nonull.platform.value_counts()

In [ ]:
(merged_gene_cn_profile_nonull.ModelID.value_counts()>1).sum()

In [ ]:
merged_segments_profile = tc.get(name='cn-latest-d8d4', version=40, file='merged_segments_profile')

In [ ]:
merged_segments_profile.head()